## Corey Solitaire
### advancecd_dataframes
#### 8/14/2020

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('figure', figsize=(12, 6))
from pydataset import data

In [2]:
"""
Test for SQL import and env.py
"""
from env import host, user, password

url = f'mysql+pymysql://{user}:{password}@{host}/employees'

In [3]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


In [4]:
# Big Test, that Passed...

pd.read_sql("""
SELECT CONCAT(employees.first_name, " ", employees.last_name) AS "Employee Name", departments.dept_name AS "Department Name", manager_name AS "Manager Name"
FROM employees
    JOIN dept_emp
        ON employees.emp_no = dept_emp.emp_no
    JOIN departments
        ON dept_emp.dept_no = departments.dept_no
    JOIN dept_manager
        ON dept_manager.dept_no = dept_emp.dept_no
-- The following subquery is a table that connects the current department manager name to their emp_no and its alias is manager_names

    JOIN(
        SELECT employees.emp_no, CONCAT(employees.first_name, " ", employees.last_name) AS manager_name
            FROM employees
                JOIN dept_manager
                    ON employees.emp_no = dept_manager.emp_no AND dept_manager.to_date > curdate()
    ) AS manager_names
WHERE dept_manager.to_date > curdate()
    AND dept_emp.to_date > curdate()
    AND dept_manager.emp_no = manager_names.emp_no;""", url)

,Employee Name,Department Name,Manager Name
0,Cristinel Bouloucos,Marketing,Vishwani Minakawa
1,Berhard McFarlin,Marketing,Vishwani Minakawa
2,Yucel Auria,Marketing,Vishwani Minakawa
3,Karoline Cesareni,Marketing,Vishwani Minakawa
4,Nikolaos Llado,Marketing,Vishwani Minakawa
...,...,...,...
240119,Aloke Wuwongse,Customer Service,Yuchang Weedman
240120,Masako Angiulli,Customer Service,Yuchang Weedman
240121,Chuanti Riesenhuber,Customer Service,Yuchang Weedman
240122,Ronghao Morrow,Customer Service,Yuchang Weedman


# Advanced Dataframes Exercises

In [5]:
"""
mpg = data('mpg') # load the dataset and store it in a variable
# data('mpg', show_doc=True) # view the documentation for the dataset
"""

"\nmpg = data('mpg') # load the dataset and store it in a variable\n# data('mpg', show_doc=True) # view the documentation for the dataset\n"

### 1. Load the mpg dataset. Read the documentation for it, and use the data to answer these questions:

In [6]:
mpg_df = data('mpg')
data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [7]:
# On average, which manufacturer has the best miles per gallon

mpg_df.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [36]:
mpg_df = mpg_df.assign(avg_mpg = (mpg_df.cty + mpg_df.hwy) /2)
mpg_df

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,avg_mpg
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,23.5
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,25.0
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,21.0
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,22.0


In [38]:
mpg_df.sort_values(by=['avg_mpg', 'manufacturer'], ascending = False)


,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,avg_mpg
222,volkswagen,new beetle,1.9,1999,4,manual(m5),f,35,44,d,subcompact,39.5
213,volkswagen,jetta,1.9,1999,4,manual(m5),f,33,44,d,compact,38.5
223,volkswagen,new beetle,1.9,1999,4,auto(l4),f,29,41,d,subcompact,35.0
197,toyota,corolla,1.8,2008,4,manual(m5),f,28,37,r,compact,32.5
196,toyota,corolla,1.8,1999,4,manual(m5),f,26,35,r,compact,30.5
...,...,...,...,...,...,...,...,...,...,...,...,...
127,jeep,grand cherokee 4wd,4.7,2008,8,auto(l5),4,9,12,e,suv,10.5
55,dodge,dakota pickup 4wd,4.7,2008,8,auto(l5),4,9,12,e,pickup,10.5
60,dodge,durango 4wd,4.7,2008,8,auto(l5),4,9,12,e,suv,10.5
66,dodge,ram 1500 pickup 4wd,4.7,2008,8,auto(l5),4,9,12,e,pickup,10.5


In [45]:
# How many different manufacturers are there?

len(mpg_df.groupby('manufacturer'))


15

In [50]:
# How many different models are there?

"""
The first step to verify count (38 Rows)
"""
#mpg_df.groupby('model').describe()

"""
Apply len function to return an answer
"""
len(mpg_df.groupby('model').describe())

38

In [52]:
# Do automatic or manual cars have better miles per gallon?
mpg_df.head()

mpg_df.sort_values(by=['avg_mpg', 'trans'], ascending = False)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,avg_mpg
222,volkswagen,new beetle,1.9,1999,4,manual(m5),f,35,44,d,subcompact,39.5
213,volkswagen,jetta,1.9,1999,4,manual(m5),f,33,44,d,compact,38.5
223,volkswagen,new beetle,1.9,1999,4,auto(l4),f,29,41,d,subcompact,35.0
197,toyota,corolla,1.8,2008,4,manual(m5),f,28,37,r,compact,32.5
100,honda,civic,1.6,1999,4,manual(m5),f,28,33,r,subcompact,30.5
...,...,...,...,...,...,...,...,...,...,...,...,...
70,dodge,ram 1500 pickup 4wd,4.7,2008,8,manual(m6),4,9,12,e,pickup,10.5
55,dodge,dakota pickup 4wd,4.7,2008,8,auto(l5),4,9,12,e,pickup,10.5
60,dodge,durango 4wd,4.7,2008,8,auto(l5),4,9,12,e,suv,10.5
66,dodge,ram 1500 pickup 4wd,4.7,2008,8,auto(l5),4,9,12,e,pickup,10.5


### 2. Joining and Merging

In [63]:
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [64]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


In [56]:
# Copy the users and roles dataframes from the examples above. What do you think a right join would look like?  

"""
pd.merge(left, right, how="type_of_join", left_on="key from the left", right_on="key from the right df")
Left = NAME of 1st Dataframe
Right = NAME of 2nd Dataframe
LEFT JOIN means keep all the USERS even if they don't have a role...
"""

users_with_roles = pd.merge(users, roles, left_on='role_id', right_on='id', how='left')
users_with_roles


,id_x,name_x,role_id,id_y,name_y
0,1,bob,1.0,1.0,admin
1,2,joe,2.0,2.0,author
2,3,sally,3.0,3.0,reviewer
3,4,adam,3.0,3.0,reviewer
4,5,jane,NaN,NaN,NaN
5,6,mike,NaN,NaN,NaN


In [65]:
# An outer join? (100 % Inclusive of Null Values)

users_with_roles_all_null = pd.merge(users, roles, left_on = 'role_id', right_on = 'id' , how = 'outer')
users_with_roles_all_null

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


In [75]:
# What happens if you drop the foreign keys from the dataframes and try to merge them?

users_test = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
})
users_test

,id,name
0,1,bob
1,2,joe
2,3,sally
3,4,adam
4,5,jane
5,6,mike


In [77]:
roles_test = pd.DataFrame({
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles_test

,name
0,admin
1,author
2,reviewer
3,commenter


In [78]:
# The merging...
no_foreign_key = pd.merge(users_test, roles_test, left_on = 'name', right_on = 'name' , how = 'outer')
no_foreign_key

,id,name
0,1.0,bob
1,2.0,joe
2,3.0,sally
3,4.0,adam
4,5.0,jane
5,6.0,mike
6,NaN,admin
7,NaN,author
8,NaN,reviewer
9,NaN,commenter


### 3. Getting data from SQL databases

In [ ]:
# Create a function named get_db_url. It should accept a 
# username, hostname, password, and database name and return a url formatted like in the examples in this lesson.


In [ ]:
# Use your function to obtain a connection to the employees database.


In [ ]:
# Once you have successfully run a query:
#  - Intentionally make a typo in the database url. What kind of error message do you see?


In [ ]:
#   -Intentionally make an error in your SQL query. What does the error message look like?
        

In [ ]:
# Read the employees and titles tables into two separate dataframes


In [ ]:
# Visualize the number of employees with each title.


In [ ]:
# Join the employees and titles dataframes together.


In [ ]:
# Visualize how frequently employees change titles.


In [ ]:
# For each title, find the hire date of the employee that was hired most recently with that title.


In [ ]:
# Write the code necessary to create a cross tabulation of the number of titles by department. 
#  (Hint: this will involve a combination of SQL and python/pandas code)


### 4. Use your get_db_url function to help you explore the data from the chipotle database. Use the data to answer the following questions:

In [ ]:
# What is the total price for each order?


In [ ]:
# What are the most popular 3 items?


In [ ]:
# Which item has produced the most revenue?
